<a href="https://colab.research.google.com/github/tomkeane07/rugby_tracker/blob/main/rugby_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')

In [ ]:
# !pip install  opencv-python==3.4.2.17
# !pip install  opencv-contrib-python==3.4.2.17

%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import imutils
import cv2
import copy
import sys
from sklearn.cluster import AgglomerativeClustering
try:
  from pyfld import FastLineDetector
except:
  ! pip install pyfld
  from pyfld import FastLineDetector
import collections
import pandas as pd

from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

from os import linesep
from scipy.spatial import distance
from itertools import chain
import matplotlib
import matplotlib.pyplot as plt
import operator

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
try:
  model
except:
  ! git clone https://github.com/ultralytics/yolov5  # clone
  ! pip install -r ./yolov5/requirements.txt  # install
  import torch
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
from pyfld.Exceptions import LineNotFound, PointChainNotFound


In [ ]:
class Point(list):
    def __init__(self, x, y):
        super().__init__([x,y])
    @property
    def x(self):
        return self[0]
    @property
    def y(self):
        return self[1]

In [ ]:
class Segment(list):
    def __init__(self, x1, y1, x2, y2):
        super().__init__([x1, y1, x2, y2])
    @property
    def x1(self):
        return self[0]
    @property
    def y1(self):
        return self[1]
    @property
    def x2(self):
        return self[2]
    @property
    def y2(self):
        return self[3]
    @property
    def angle(self):
        return np.arctan2(self.y2 - self.y1, self.x2 - self.x1)
    @property
    def length(self):
        return np.sqrt((self.x2-self.x1)**2 + (self.y2-self.y1)**2)
    @property
    def y_length(self):
        return abs(self.y2-self.y1)
    @property
    def x_length(self):
        return abs(self.x2-self.x1)
    @property
    def p1(self):
        return Point(self.x1, self.y1)
    @property
    def p2(self):
        return Point(self.x2, self.y2)
    def swap_x(self):
        self[0], self[2] = self[2], self[0]
    def swap_y(self):
        self[1], self[3] = self[3], self[1]

In [ ]:
class FastLineDetector:
    def __init__(self, length_threshold=10, distance_threshold=1.414213562,
                canny_th1=50.0, canny_th2=50.0, canny_aperture_size=3, do_merge=False,
                xlen_threshold=0, ylen_threshold=0):
        """
        Parameters
        ----------
        length_threshold : int or float, default 10
            Segment shorter than this will be discarded.
        distance_threshold : int or float, default 1.41421356
            A point placed from a hypothesis line segment farther than this will be regarded as an outlier.
        canny_th1 : int or float, default 50
            First threshold for hysteresis procedure in cv2.canny().
        canny_th2 : int or float, default 50
            Second threshold for hysteresis procedure in cv2.canny().
        canny_aperture_size : int, default 3
            Aperturesize for the sobel operator in cv2.canny().
            If zero, Canny() is not applied and the input image is taken as an edge image.
        do_merge : bool, default False
            If true, incremental merging of segments will be perfomred.
        xlen_threshold : int or float, default 0
            Segment shorter than this along x-axis will be discarded.
        ylen_threshold : int or float, default 0
            Segment shorter than this along y-axis will be discarded.
        """
        if (length_threshold < 0):
            raise ValueError("length_threshold must not be negative.")
        if (distance_threshold < 0):
            raise ValueError("distance_threshold must not be negative.")
        if (canny_th1 < 0):
            raise ValueError("canny_th1 must not be negative.")
        if (canny_th2 < 0):
            raise ValueError("canny_th2 must not be negative.")
        if (xlen_threshold < 0):
            raise ValueError("xlen_threshold must not be negative.")
        if (ylen_threshold < 0):
            raise ValueError("ylen_threshold must not be negative.")

        self.length_threshold = length_threshold
        self.distance_threshold = distance_threshold
        self.canny_th1 = canny_th1
        self.canny_th2 = canny_th2
        self.canny_aperture_size = canny_aperture_size
        self.do_merge = do_merge
        self.xlen_threshold = xlen_threshold
        self.ylen_threshold = ylen_threshold

    def detect(self, image):
        """
        Detect lines in the input image.

        Parameters
        ----------
        image : numpy ndarray
            A grayscale input image (dtype=np.uint8).

        Reterns
        -------
        lines : numpy ndarray
            A vector of specifying the beginning and ending point of a line.
            Where vector is (x1, y1, x2, y2), point 1 is the start, point 2 is the end.
            Returned lines are directed so that the brighter side is placed on left.
        points : numpy ndarray
            Pixels used in line detection.
        """
        image = np.array(image)
        segments, points = self.line_detection(image)
        if segments == []:
            raise LineNotFound("The image has no line segments")
        return np.array(np.array(segments), dtype="object").T, np.array(points,dtype="object")

    def line_detection(self, src):
        self._h, self._w = src.shape
        if self.canny_aperture_size == 0:
            canny = src
        else:
            canny = cv2.Canny(image=src, threshold1=self.canny_th1, threshold2=self.canny_th2, apertureSize=self.canny_aperture_size)

        canny[:5, :5] = 0
        canny[self._h-5:, self._w-5:] = 0

        points_all = []
        segments_all = []
        if np.all(canny == 0):
            return segments_all, points_all

        for r in range(self._h):
            for c in range(self._w):
                if canny[r,c] == 0:
                    continue
                pt = Point(c,r)
                points, canny = self.get_chained_points(canny, pt)

                if len(points) - 1 < self.length_threshold:
                    points = []
                    continue

                segments = self.extract_segments(points, xmin=0, xmax=self._w-1, ymin=0, ymax=self._h-1)

                if len(segments) == 0:
                    points = []
                    continue

                for seg in segments:
                    if (seg.length < self.length_threshold) or (seg.x_length < self.xlen_threshold) or (seg.y_length < self.ylen_threshold):
                        continue
                    if (seg.x1 <= 4 and seg.x2 <= 4) or (seg.y1 <= 4 and seg.y2 <= 4) or (seg.x1 >= self._w-5 and seg.x2 >= self._w-5) or (seg.y1 >= self._h-5 and seg.y2 >= self._h-5):
                        continue
                    segments_all.append(seg)
                    points_all.append(np.array(points))
                points = []
                segments = []

        if self.do_merge is False:
            return segments_all, points_all

        ith = len(segments_all) - 1
        jth = ith - 1
        while (ith >= 1 or jth >= 0):
            seg1 = segments_all[ith]
            seg2 = segments_all[jth]
            seg_merged = self.merge_segments(seg1, seg2)
            if seg_merged is None:
                jth -= 1
            else:
                segments_all[jth] = seg_merged
                del segments_all[ith]
                ith -= 1
                jth = ith - 1
            if jth < 0:
                ith -= 1
                jth = ith - 1
        return segments_all, points_all

    def get_point_chain(self, img):
        """
        Parameters
        ----------
        img : numpy ndarray
            Input edge image.

        Reterns
        -------
        point_chain : list
            Extracted point chain.
        """
        point_chain = []
        if np.all(img == 0):
            raise PointChainNotFound("The image has no point chain")

        # rs, cs = np.where(img != 0)
        # for r, c in zip(rs, cs):
        for r in range(img.shape[0]):
            for c in range(img.shape[1]):
                # Skip for non-seeds
                if img[r,c] == 0:
                    continue
                # Found seeds
                pt = Point(c,r)
                # Get point chain
                points, img = self.get_chained_points(img, pt)
                point_chain.append(points)
        return point_chain

    def get_chained_points(self, img, pt):
        """
        Parameters
        ----------
        img : numpy ndarray
            Input edge image.
        pt : Point
            Start point to search chained points.

        Reterns
        -------
        points : list of Point
            Extracted chained points.
        img : numpy ndarray
            Points removed image.
        """
        if img[pt.y, pt.x] == 0:
            raise ValueError("img[pt.y, pt.x] must not be 0")
        points = [pt]
        img[pt.y, pt.x] = 0

        direction = 0.0
        step = 0
        getting = True
        while(getting):
            pt, direction = self.get_chained_point(img, pt, direction, step)
            if pt is None:
                getting = False
                break
            points.append(pt)
            step += 1
            img[pt.y, pt.x] = 0
        return points, img

    def get_chained_point(self, img, pt, direction, step):
        """
        Find the neighboring edge point.
        Edge points closer to the direction are given priority.
        However, when step=0, it does not depend on the direction.
        Returns (None, None) if no edge point is found.

        Parameters
        ----------
        img : numpy ndarray
            Input edge image.
        pt : Point
            Start point to search the neighboring edge point.
        direction : int or float
            Previous search direction.
        step : int
            Current number of searches.

        Reterns
        -------
        point : Point
            Extracted neighbor chained point.
        direction : int or float
            Recalculated direction.
        """
        def direction_fixer(dir):
            if dir <= 180:
                return dir
            return dir - 360
        indices = {0:(1,1), 45:(1,0), 90:(1,-1), 135:(0,-1),
                   180:(-1,-1), 225:(-1,0), 270:(-1,1), 315:(0,1)} # Clockwise from lower right

        min_dir_diff = 315.0
        for i in (0, 45, 90, 135, 180, 225, 270, 315):
            ci = pt.x + indices[i][1]
            ri = pt.y + indices[i][0]
            if ri < 0 or ri == img.shape[0] or ci < 0 or ci == img.shape[1]:
                continue
            if img[ri, ci] == 0:
                continue
            if step == 0:
                chained_pt = Point(ci, ri)
                direction = direction_fixer(i)
                return chained_pt, direction
            if step > 0:
                curr_dir = direction_fixer(i)
                dir_diff = abs(curr_dir - direction)
                dir_diff = abs(direction_fixer(dir_diff))
                if dir_diff <= min_dir_diff:
                    min_dir_diff = dir_diff
                    consistent_pt = Point(ci, ri)
                    consistent_direction = direction_fixer(i)

        if min_dir_diff < 90.0:
            chained_pt = consistent_pt
            direction = (direction * step + consistent_direction) / (step + 1)
            return chained_pt, direction
        return None, None

    def extract_segments(self, points, xmin=0, xmax=None, ymin=0, ymax=None):
        """
        Extract segments from point chain.

        Parameters
        ----------
        points : list of Point
            Point chain for extraction.
        xmin : int or float, default 0
            Minimum x. Segment's x smaller than xmin are rounded to xmin.
        xmax : int or float, optional
            Minimum x. Segment's x larger than xmax are rounded to xmax.
        ymin : int or float, default 0
            Minimum y. Segment's y smaller than ymin are rounded to ymin.
        ymin : int or float, optional
            Minimum y. Segment's y smaller than ymax are rounded to ymax.
        """
        segments = []
        total = len(points)
        skip = 0
        for i in range(total-self.length_threshold):
            if skip > 0:
                skip -= 1
                continue

            ps = points[i]
            pe = points[i+self.length_threshold]

            is_line = True
            l_points = [ps]

            for j in range(1, self.length_threshold):
                pt = Point(points[i+j].x, points[i+j].y)
                dist = self.dist_point_line(ps, pe, pt)
                if dist > self.distance_threshold:
                    is_line = False
                    break
                l_points.append(pt)

            # Line check fail, test next point
            if is_line is False:
                continue

            l_points.append(pe)

            vx, vy, x, y = cv2.fitLine(np.array(l_points).astype(np.float32), cv2.DIST_L2, 0, 0.01, 0.01)
            a = Point(x.item(), y.item())
            b = Point(x.item() + vx.item(), y.item() + vy.item())
            ps = self.get_incident_point(a, b, ps, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

            # Extending line
            for j in range(self.length_threshold+1, total-i):
                pt = Point(points[i+j].x, points[i+j].y)
                dist = self.dist_point_line(a, b, pt)
                if dist > self.distance_threshold:
                    vx, vy, x, y = cv2.fitLine(np.array(l_points).astype(np.float32), cv2.DIST_L2, 0, 0.01, 0.01)
                    a = Point(x.item(), y.item())
                    b = Point(x.item() + vx.item(), y.item() + vy.item())
                    dist2nd = self.dist_point_line(a, b, pt)
                    if dist2nd > self.distance_threshold:
                        j -= 1
                        break
                pe = pt
                l_points.append(pt)
            vx, vy, x, y = cv2.fitLine(np.array(l_points).astype(np.float32), cv2.DIST_L2, 0, 0.01, 0.01)
            a = Point(x.item(), y.item())
            b = Point(x.item() + vx.item(), y.item() + vy.item())
            e1 = self.get_incident_point(a, b, ps, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
            e2 = self.get_incident_point(a, b, pe, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
            segments.append(Segment(e1.x, e1.y, e2.x, e2.y))
            if self.length_threshold == 1:
                j = 1
            skip = j
        return segments

    @staticmethod
    def dist_point_line(p1, p2, p3):
        """
        Calcurate the distance between the line (p1 to p2) and point (p3).
        """
        u = np.array([p2.x - p1.x, p2.y - p1.y])
        v = np.array([p3.x - p1.x, p3.y - p1.y])
        l = abs(np.cross(u, v)/ np.linalg.norm(u))
        return l

    @staticmethod
    def get_incident_point(p1, p2, p3, xmin=0, xmax=None, ymin=0, ymax=None):
        """
        Calcurate the incident point on the line (p1 to p2) from point (p3).
        """
        l = np.cross([p1.x, p1.y, 1.0], [p2.x, p2.y, 1.0])
        lh = [l[0], l[1], 0.0]
        xk = [p3.x, p3.y, 1.0]

        lk = np.cross(xk, lh)
        xk = np.cross(lk, l)

        xk = xk/xk[2]

        if xk[0] < xmin:
            pt_x = xmin
        elif xmax is not None and xk[0] > xmax:
            pt_x = xmax
        else:
            pt_x = xk[0]

        if xk[1] < ymin:
            pt_y = ymin
        elif ymax is not None and xk[1] > ymax:
            pt_y = ymax
        else:
            pt_y = xk[1]

        return Point(pt_x, pt_y)

    def adjust_left_of_segment_to_be_higher(self, src, seg, gap=1, num_points=10):
        """
        Adjust the left side of the segment to be higher value.
        The line segment is divided into num_points and evaluated by the average
        of the values at vertically separated positions by a gap.

        Parameters
        ----------
        src : numpy ndarray
            Source image
        seg : Segment
            Segment to adjust.
        gap : int or float, default 1
            Distance from the segment when getting the value.
        num_points : int, default 10
            Divide the line segment into num_points and get the value.

        Reterns
        -------
        seg : Segment
            Adjusted segment.
        """
        if seg.x1 == 0 and seg.x2 == 0 and seg.y1 == 0 and seg.y2 == 0:
            return seg
        start = seg.p1
        end = seg.p2

        dx = end.x - start.x
        dy = end.y - start.y

        x10 = np.linspace(seg.x1, seg.x2, num_points)
        y10 = np.linspace(seg.y1, seg.y2, num_points)

        x10L = np.round(x10 + gap * np.cos(seg.angle + np.pi/2)).astype(int)
        y10L = np.round(y10 + gap * np.sin(seg.angle + np.pi/2)).astype(int)
        x10L_inboard = np.logical_and(x10L >= 0, x10L < src.shape[1])
        y10L_inboard = np.logical_and(y10L >= 0, y10L < src.shape[0])
        x10L = x10L[x10L_inboard * y10L_inboard]
        y10L = y10L[x10L_inboard * y10L_inboard]
        left = src[y10L, x10L].mean()

        x10R = np.round(x10 - gap * np.cos(seg.angle + np.pi/2)).astype(int)
        y10R = np.round(y10 - gap * np.sin(seg.angle + np.pi/2)).astype(int)
        x10R_inboard = np.logical_and(x10R >= 0, x10R < src.shape[1])
        y10R_inboard = np.logical_and(y10R >= 0, y10R < src.shape[0])
        x10R = x10R[x10R_inboard * y10R_inboard]
        y10R = y10R[x10R_inboard * y10R_inboard]
        right = src[y10R, x10R].mean()

        if right > left:
            seg.swap_x()
            seg.swap_y()

        return seg

    def merge_segments(self, seg1, seg2):
        """
        Merge segments if some conditions are passed.

        Parameters
        ----------
        seg1 : Segment
            Segment 1.
        seg2 : Segment
            Segment 2.

        Reterns
        -------
        seg_merged : Segment
            Merged segment.
        """
        p1 = Point(seg1.x1, seg1.y1)
        p2 = Point(seg1.x2, seg1.y2)

        seg1mid = Point((seg1.x1 + seg1.x2)/2, (seg1.y1 + seg1.y2)/2)
        seg2mid = Point((seg2.x1 + seg2.x2)/2, (seg2.y1 + seg2.y2)/2)

        seg1len = np.sqrt((seg1.x1 - seg1.x2)**2 + (seg1.y1 - seg1.y2)**2)
        seg2len = np.sqrt((seg2.x1 - seg2.x2)**2 + (seg2.y1 - seg2.y2)**2)

        mid_dist = np.sqrt((seg1mid.x - seg2mid.x)**2 + (seg1mid.y - seg2mid.y)**2)
        ang_diff = abs(seg1.angle - seg2.angle)

        dist = self.dist_point_line(p1, p2, seg2mid)

        if dist <= self.distance_threshold * 2 and mid_dist <= seg1len/2 + seg2len/2 + 20 and\
           ang_diff <= np.deg2rad(5.0):
           seg_merged = self.merge_lines(seg1, seg2)
           return seg_merged
        else:
            return None

    @staticmethod
    def merge_lines(seg1, seg2):
        """
        Merge line segments.

        Parameters
        ----------
        seg1 : Segment
            Segment 1.
        seg2 : Segment
            Segment 2.

        Reterns
        -------
        seg_merged : Segment
            Merged segment.
        """
        ax = seg1.x1
        ay = seg1.y1
        bx = seg1.x2
        by = seg1.y2

        cx = seg2.x1
        cy = seg2.y1
        dx = seg2.x2
        dy = seg2.y2

        dlix = bx - ax
        dliy = by - ay
        dljx = dx - cx
        dljy = dy - cy

        li = np.sqrt(dlix**2 + dliy**2)
        lj = np.sqrt(dljx**2 + dljy**2)

        xg = (li * (ax + bx) + lj * (cx + dx)) / (2 * (li + lj))
        yg = (li * (ay + by) + lj * (cy + dy)) / (2 * (li + lj))

        if dlix == 0:
            thi = np.pi/2
        else:
            thi = np.arctan(dliy / dlix)

        if dljx == 0:
            thj = np.pi/2
        else:
            thj = np.arctan(dljy / dljx)

        if np.abs(thi - thj) <= np.pi / 2:
            thr = (li * thi + lj * thj) / (li + lj)
        else:
            tmp = thj - np.pi * (thj / abs(thj))
            thr = li * thi + lj * tmp
            thr /= (li + lj)

        axg = (ay - yg) * np.sin(thr) + (ax - xg) * np.cos(thr)
        bxg = (by - yg) * np.sin(thr) + (bx - xg) * np.cos(thr)
        cxg = (cy - yg) * np.sin(thr) + (cx - xg) * np.cos(thr)
        dxg = (dy - yg) * np.sin(thr) + (dx - xg) * np.cos(thr)

        delta1xg = min([axg, bxg, cxg, dxg])
        delta2xg = max([axg, bxg, cxg, dxg])

        delta1x = delta1xg * np.cos(thr) + xg
        delta1y = delta1xg * np.sin(thr) + yg
        delta2x = delta2xg * np.cos(thr) + xg
        delta2y = delta2xg * np.sin(thr) + yg

        seg_merged = Segment(delta1x, delta1y, delta2x, delta2y)
        return seg_merged

In [ ]:
class ImgMorphOperator:
  def __init__(self):
    # self.kernel2 = np.ones((2,2), np.uint8)
    self.kernel3 = np.ones((3,3), np.uint8)
    # self.kernel4 = np.ones((4,4), np.uint8)
    # self.kernel5 = np.ones((5,5), np.uint8)
    # self.kernel6 = np.ones((6,6), np.uint8)
    self.kernel8 = np.ones((8,8), np.uint8)
    # self.kernel10 = np.ones((10,10), np.uint8)
    # self.kernel15 = np.ones((15,15), np.uint8)
    # self.kernel20 = np.ones((20,20), np.uint8)
    self.kernel30 = np.ones((30,30), np.uint8)
    self.kernel50 = np.ones((50,50), np.uint8)
    # self.kernel1x100 = np.ones((1,100), np.uint8)
    # self.kernel1x200 = np.ones((1,200), np.uint8)
    # self.kernel1x500 = np.ones((1,500), np.uint8)
    # self.kernel15x1 = np.ones((15,1), np.uint8)
    self.kernel5x50 = np.ones((5,50), np.uint8)
    self.kernel15x2 = np.ones((15,2), np.uint8)
    self.kernel15x1 = np.ones((15,1), np.uint8)
    self.kernel5x30 = np.ones((5,30), np.uint8)
    # self.kernel60 = np.ones((60,60), np.uint8)
    self.kernel75 = np.ones((75, 75), np.uint8)
    # self.kernel85 = np.ones((85, 85), np.uint8)
    # self.kernel100 = np.ones((100,100), np.uint8)


  def run(self, mask):
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel8)
    # mask = cv2.dilate(mask, self.kernel30, 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel75)
    mask = cv2.dilate(mask, self.kernel15x1, 1)
    mask = cv2.dilate(mask, self.kernel15x1, 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel75)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    # mask = cv2.erode(mask, self.kernel1x30, 1)
    # mask = cv2.erode(mask, self.kernel1x30, 1)
    # mask = cv2.erode(mask, self.kernel1x30, 1)
    # mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel30)
    return mask

In [ ]:
class PitchIsolator:
  def __init__(self, imgMorphOperator=None):
    self.imo = ImgMorphOperator() if imgMorphOperator==None else imgMorphOperator
  # (70,115,85), (80,255,180)

  def findPitch(self, image, range=[(0, 100, 60), (100, 255, 155)]):
    mask = cv2.inRange(image, lowerb=range[0], upperb=range[1])
    yg = cv2.bitwise_not(cv2.inRange(image, lowerb=(65,145,125), upperb=(92,175,155)))
    dg = cv2.bitwise_not(cv2.inRange(image, lowerb=(65,93,56), upperb=(87,125,96)))
    mask = cv2.bitwise_and(mask, yg)
    mask = cv2.bitwise_and(mask, dg)
    # cv2_imshow(dg)
    # cv2_imshow(mask)
    mask = self.imo.run(mask)
    mask = self.cropImg(mask)
    return mask

  def removeNonPitch(self, image, mask):
    # apply mask
    return cv2.bitwise_and(image, image, mask=mask)

  def isoLines(self, image, range=[(110,150,150), (130,255,255)]):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lowerb=range[0], upperb=range[1])
    return cv2.bitwise_and(image, image, mask=mask)


  def cropImg(self, image,tol=1):
      # img is 2D image data
      # tol  is tolerance
      self.cropMask = image>tol
      return image[np.ix_(self.cropMask.any(1),self.cropMask.any(0))]


  def isolatePitch(self, image):
    mask = self.findPitch(cv2.convertScaleAbs(copy.deepcopy(image), alpha=1.5, beta=-30))
    frame_masked = self.removeNonPitch(image[np.ix_(self.cropMask.any(1),self.cropMask.any(0))], mask)
    # cv2_imshow(frame_masked)
    return frame_masked

In [ ]:
class LineMerger:
  def __init__(self, lco):
    self.lco = lco
    self.cornerPoints = np.empty(shape=(0,3))

  def lengthPriority(self, lineGroup):
    lll = copy.deepcopy(lineGroup)
    for ln in lineGroup:
      for i in range(int(self.lco.dist(ln=ln[0])//10)):
        lll = np.append(lll, [ln], axis=0)
    return lll

  def getCornerPoints(self, linesID):
    self.CornerPoints = {'ovrMax':100, 'ovrMin':100}
    for lnType in np.unique(linesID[:,1]):
      indices = np.where((np.array(linesID[:,1])==lnType))[0]
      ys = [row[0][[1,3]] for row in linesID[indices]]
      ys = list(chain.from_iterable(ys))
      minIndex, maxIndex = np.argmin(ys), np.argmax(ys)
      if ys[minIndex] < self.CornerPoints['ovrMin']:
        self.CornerPoints['ovrMin'] = ys[minIndex]
      if ys[minIndex] > self.CornerPoints['ovrMax']:
        self.CornerPoints['ovrMax'] = ys[minIndex]
      minPoint = [linesID[indices][minIndex//2][0][minIndex%2], linesID[indices][minIndex//2][0][minIndex%2+1]]
      maxPoint = [linesID[indices][maxIndex//2][0][maxIndex%2], linesID[indices][maxIndex//2][0][maxIndex%2+1]]
      self.CornerPoints[lnType] = {'minPoint':minPoint, 'maxPoint':maxPoint}

  def refineLines(self, lineGroup, minLenFactor=0.3):
    delLinesIndex = []
    longestLen  = np.max([self.lco.dist(self.lco.points(x)) for x in lineGroup[:,0]])
    shortestLen = np.min([self.lco.dist(self.lco.points(x)) for x in lineGroup[:,0]])
    defaultDist = longestLen*minLenFactor
    for i in range(len(lineGroup)):
        ln =  lineGroup[i][0]
        pt1,pt2 = self.lco.points(ln)
        dist = self.lco.dist((pt1,pt2))
        if dist < defaultDist:
          delLinesIndex.append(i)
    return delLinesIndex

  def remove_outliers_bis(self, arr, k=1.4, lean=''):
    delIndex, mask=[], []
    arr = np.array(arr)
    mask = np.ones((arr.shape[0],), dtype=bool)
    mu, sigma = np.mean(arr, axis=0), np.std(arr, axis=0, ddof=1)
    if lean=='up':
      mu[-1]-=np.abs(sigma[-1])*1.5
    elif  lean=='down':
      mu[-1]+=np.abs(sigma[-1])*1.5
    nCols=arr.shape[1]
    for j in range(nCols):
        col = arr[:, j]
        keep = np.abs((col[mask] - mu[j]) / sigma[j]) < k
        mask[mask] &= keep
        if len(arr[mask])<=4:
          if k > 5:
            print("quit")
            return arr, []
          return self.remove_outliers_bis(arr, k=k*2)
        for i in range(len(keep)):
          if not keep[i]:
            delIndex = np.append(delIndex, i)
    delIndex = np.unique(delIndex)
    return arr[mask], delIndex.astype(int)

  def mergeLineType(self, lineGroup, lnType):
    delIndex = self.refineLines(lineGroup)
    lineGroup = np.delete(lineGroup, delIndex, axis=0)
    eqs=[]
    for ln in self.lengthPriority(lineGroup):
      eqs.append(self.lco.cartesianize(ln))
    if lnType=="halfway":
      k=1
      lean=''
    elif lnType=="nearSideLine":
      k=0.3
      lean='down'
    elif lnType=="farSideLine":
      k=0.3
      lean='down'
    elif lnType=="tryLineL" or lnType=="tryLineR":
      k=0.2
      lean='down'
    elif lnType=="22L" or lnType=="22R":
      k=0.3
      lean=''
    else:
      k=1
      lean=''
    eqs, delIndex = self.remove_outliers_bis(eqs,k=k, lean=lean)
    i,j=0,0
    while  i <len(delIndex)-1:
      j=delIndex[i]
      if j >= len(lineGroup):break
      lineGroup = np.delete(lineGroup,j, axis=0)
      i+=1
    minmaxX = [np.min(np.array([pts[[0,2]] for pts in lineGroup[:,0]]).flatten()), np.max(np.array([pts[[0,2]] for pts in lineGroup[:,0]]).flatten())]
    minmaxY = [np.min(np.array([pts[[1,3]] for pts in lineGroup[:,0]]).flatten()), np.max(np.array([pts[[1,3]] for pts in lineGroup[:,0]]).flatten())]
    newLine = [
               [minmaxX[0], np.poly1d(np.mean(eqs, axis=0))(minmaxX[0]),
                minmaxX[1], np.poly1d(np.mean(eqs, axis=0))(minmaxX[1])],
                lnType,
                np.mean(eqs, axis=0)
              ]
    return newLine

  def run(self, linesID):
    delIndex = np.where((np.array(linesID[:,1])=="other")|(np.array(linesID[:,1])=="UpPitchLine"))[0]
    linesID = np.delete(linesID, delIndex, axis=0)
    self.getCornerPoints(linesID)
    lnTypes = np.unique(linesID[:,1])
    newLines = []
    for lnType in lnTypes:
      self.indices = np.where((np.array(linesID[:,1])==lnType))[0]
      newLine = self.mergeLineType(linesID[self.indices], lnType)
      newLines.append(newLine)
    return newLines

In [ ]:
class LineClassifier:
  def __init__(self, lco):
    self.lco = lco

  def fixNearCornerUpFieldLines(self, lineTypeSlopes, linesID):
    if lineTypeSlopes['22L'] > -0.6 and lineTypeSlopes['22L']<0:
      for ln in linesID:
        if ln[1]=="tryLineR":
          ln[1]="UpPitchLine"
    if lineTypeSlopes['22R'] < 0.6 and lineTypeSlopes['22R']>0:
      for ln in linesID:
        if ln[1]=="tryLineL":
          ln[1]="UpPitchLine"
    return linesID


  def lookLeftRight(self, linesID):
    self.lookRight = self.lineTypeLengthSum['22R'] > self.lineTypeLengthSum['22L']*2
    self.lookLeft  = self.lineTypeLengthSum['22L'] > self.lineTypeLengthSum['22R']*2
    if self.lookRight:
      linesID = np.delete(
          linesID, np.where((linesID[:,1]=='22L')|(linesID[:,1]=='tryLineL'))[0],axis=0
      )
    elif self.lookLeft:
      linesID = np.delete(
          linesID, np.where((linesID[:,1]=='22R')|(linesID[:,1]=='tryLineR'))[0],axis=0
      )
    return linesID


  def getNumCollinears(self, linesID, group="farSideLine"):
    rows = np.array([row for row in linesID])
    farSideIndices = np.where((np.array(rows[:,1]) == 'farSideLine'))
    for ln in linesID:
      if ln[1] == group:
        ln =np.append(ln, len(self.lco.getCollinears(ln[0], [linesID[index] for index in farSideIndices])))
    return linesID

  def hidegoalPostLines(self, linesID, lineTypeLengthSum):
    delIndexs = []
    if lineTypeLengthSum['tryLineL'] > lineTypeLengthSum['halfway']*5 or lineTypeLengthSum['tryLineR'] > lineTypeLengthSum['halfway']*5:
      for i in range(len(linesID)):
        if linesID[i][1] == 'halfway':
          delIndexs.append(i)
      linesID = np.delete(linesID, [delIndexs],0)
    return linesID

  def cleanNearSideLine(self, linesID, lineLengths):
    delIndexs = []
    minLength = np.max(lineLengths)*0.1
    for i in range(len(linesID)):
      if linesID[i][1] == 'nearSideLine':
        tooShort=lineLengths[i] < minLength
        thisSlope = self.lco.getSlope(ln=linesID[i][0])
        wrongSign = ((self.lookRight and thisSlope/np.mean(self.lineTypeSlopes['22R'])>0) or
                    (self.lookLeft and thisSlope/np.mean(self.lineTypeSlopes['22L'])>0))
        tooSlopedAtHalfway = self.nearHalfWay and np.abs(thisSlope) > 0.1
        if wrongSign or tooSlopedAtHalfway:
          delIndexs.append(i)
    if len(delIndexs) > 0:
      linesID, lineLengths = np.delete(linesID, [delIndexs],axis=0), np.delete(lineLengths, [delIndexs],axis=0)
    return linesID, lineLengths


  def cleanFarSideLine(self, linesID, lineLengths):
    delIndexs = []
    minLength = np.max(lineLengths)*0.1
    for i in range(len(linesID)):
      if linesID[i][1] == 'farSideLine':
        tooShort=lineLengths[i] < minLength
        wrongSign = ((self.lookRight and self.lco.getSlope(ln=linesID[i][0])/np.mean(self.lineTypeSlopes['22R'])>0) or
                    (self.lookLeft and self.lco.getSlope(ln=linesID[i][0])/np.mean(self.lineTypeSlopes['22L'])>0))
        if wrongSign:
          delIndexs.append(i)
    linesID, lineLengths = np.delete(linesID, [delIndexs],0), np.delete(lineLengths, [delIndexs],0)
    return linesID, lineLengths

  def cleanHalfwayLine(self, linesID):
    meanSlope = np.mean(self.lineTypeSlopes['halfway'])
    stdSlope = np.std(self.lineTypeSlopes['halfway'])
    delIndex = []
    for i in range(len(np.array(linesID))):
      ln = linesID[i]
      if ln[1]=="halfway":
        if ((max(self.lineTypeLengthSum.values()) > self.lineTypeLengthSum['halfway']*5) or
            np.abs(self.lco.getSlope(ln=linesID[i][0]) - meanSlope)>2*stdSlope):
          delIndex.append(i)
    linesID = np.delete(linesID, [delIndex],0)
    return linesID

  def lineIsHighAndSlopeInRange(self, lineHeightsInPic, linesID, i,  rangeSlope=0.13):
    thisSlope = self.lco.getSlope(ln=linesID[i][0])
    if self.lookLeft or self.lookRight:
      slopeInRange = np.isclose(thisSlope/rangeSlope, 0.33, atol=0.085)
    else:
      slopeInRange = np.isclose(thisSlope, 0, atol=0.02)
    return lineHeightsInPic[i] < np.max(lineHeightsInPic)*0.1 and  slopeInRange

  def lineIsLowAndFlatterThan(self, lineHeightsInPic, linesID, i,  slope=0.4):
    return lineHeightsInPic[i] > np.max(lineHeightsInPic)*0.78 and np.abs(np.abs(self.lco.getSlope(ln=linesID[i][0])) - slope) > 0.02

  def slopeOutlier(self, slopes, meanSlope, std, i, stdF=3):
    np.abs(slopes[i] - meanSlope) > stdF*std

  def removeOutliers(self, slopes, meanSlope, std, linesID, lineIndices):
    delIndex,i = [],0
    while True and len(lineIndices)>0:
      index = lineIndices[i]
      height = (linesID[i][0][1]+linesID[i][0][3])
      if  self.slopeOutlier(slopes, meanSlope, std, i):
        delIndex.append(index)
      i+=1
      if i >= len(lineIndices)-1:
        break
    return delIndex

  def detectTryLines(self, linesID):
    delIndex = []
    upPitchIndices = np.where((linesID[:,1]=='UpPitchLine'))[0]
    if self.lookRight:
      lnType = '22R'
    elif self.lookLeft:
      lnType = '22L'
    else:
      return linesID
    lines22 = linesID[np.where((linesID[:,1]==lnType))[0]]
    mean22Slope = np.mean([self.lco.getSlope(ln[0]) for ln in lines22])
    for i in upPitchIndices:
      ln = linesID[i]
      slope = self.lco.getSlope(ln[0])
      if slope*mean22Slope < 0 or not np.isclose(slope/mean22Slope, 0.5, atol=0.1):
        delIndex.append(i)
        continue
      else:
        ln[1]='tryLine'+lnType[-1]
    linesID = np.delete(linesID, delIndex, axis=0)
    return linesID

  def detectFarSideLine(self, linesID, upPitchLinesIndices):
    lineIndices = []
    slopes = []
    lineHeightsInPic = np.zeros(shape=(len(linesID)))
    for i in upPitchLinesIndices:
      lineHeightsInPic[i] = linesID[i][0][1]+linesID[i][0][3]
    for i in upPitchLinesIndices:
      if self.lineIsHighAndSlopeInRange(lineHeightsInPic, linesID, i, rangeSlope=np.mean(self.lineTypeSlopes['nearSideLine'])):
        linesID[i][1] = "farSideLine"
        slopes.append(self.lco.getSlope(linesID[i][0]))
        lineIndices.append(i)
    meanSlope = np.mean(slopes)
    self.lineTypeSlopes['farSideLine'] = meanSlope
    self.nearHalfWay=np.isclose(meanSlope, 0, atol=0.06, equal_nan=False)
    std = np.std(slopes)
    delIndex = self.removeOutliers(slopes, meanSlope, std, linesID, lineIndices)
    linesID = np.delete(linesID, [delIndex],0)
    return linesID

  def detectNearSideLine(self, linesID, upPitchLinesIndices):
    nearSideLineIndices = []
    nearSlopes = []
    lineHeightsInPic = np.zeros(shape=(len(linesID)))
    for i in upPitchLinesIndices:
      lineHeightsInPic[i] = linesID[i][0][1]+linesID[i][0][3]
    for i in upPitchLinesIndices:
      if self.lineIsLowAndFlatterThan(lineHeightsInPic, linesID, i, 0.4):
        linesID[i][1] = "nearSideLine"
        nearSlopes.append(self.lco.getSlope(linesID[i][0]))
        nearSideLineIndices.append(i)
    nearMeanSlope = np.mean(nearSlopes)
    self.lineTypeSlopes['nearSideLine'] = nearMeanSlope
    std = np.std(nearSlopes)
    delIndex,i = [],0
    while True and len(nearSideLineIndices)>0:
      index = nearSideLineIndices[i]
      if np.abs(nearSlopes[i] - nearMeanSlope) > std:
        delIndex.append(index)
      i+=1
      if i > len(nearSideLineIndices)-1:
        break
    linesID = np.delete(linesID, [delIndex],0)
    nearSideLineIndices=np.where((linesID[:,1]=="nearSideLine"))[0]
    self.lineTypeSlopes['nearSideLine'] = np.mean([self.lco.getSlope(ln[0]) for ln in linesID[nearSideLineIndices]])
    return linesID


  def differUpPitchLines(self, linesID):
    slopes = []
    upPitchLinesIndices = self.lco.indicesOfUpPitchLine(linesID)
    linesID = self.detectNearSideLine(linesID, upPitchLinesIndices)
    upPitchLinesIndices = self.lco.indicesOfUpPitchLine(linesID)
    linesID = self.detectFarSideLine(linesID, upPitchLinesIndices)
    return linesID

  def setLineTypeMetrics(self, linesID):
    self.lineTypeLengthSum =  {'UpPitchLine': 0, 'halfway': 0, 'other': 0, '22L': 0, 'tryLineL':0, 'tryLineR':0, '22R':0}
    self.lineTypeSlopes =     {'UpPitchLine': [], 'halfway': [], 'other': [], '22L': [], 'tryLineL':[], 'tryLineR':[], '22R':[]}
    for i in range(len(linesID)):
      dist = self.lco.dist(ln=linesID[i][0])
      lineType = linesID[i][1]
      if lineType in self.lineTypeLengthSum:
        self.lineTypeLengthSum[lineType] += dist
        self.lineTypeSlopes[lineType] = np.append(self.lineTypeSlopes[lineType], np.array([self.lco.getSlope(linesID[i][0])]), axis=0)
      else:
        self.lineTypeLengthSum[lineType] = dist
        self.lineTypeSlopes[lineType] = np.array([self.lco.getSlope(linesID[i][0])])

  def cleanTryLines(self, linesID):
    for dir in ['L','R']:
      delIndex = []
      indices = np.where(linesID[:,1]=='tryLine'+dir)[0]
      mean22Slope=np.mean(self.lineTypeSlopes['22'+dir])
      for i in indices:
        thisSlope = self.lco.getSlope(ln=linesID[i][0])
        if not np.isclose(np.abs((mean22Slope-thisSlope)/mean22Slope), 0.46, atol=0.05):
          delIndex.append(i)
      linesID = np.delete(linesID, delIndex, axis=0)
    return linesID



  def classifyLines(self, lines):
    linesID = []
    lineLengths = []
    self.lineTypeLengthSum =  {'UpPitchLine': 0, 'halfway': 0, 'other': 0, '22L': 0, 'tryLineL':0, 'tryLineR':0, '22R':0}
    self.lineTypeSlopes =     {'UpPitchLine': [], 'halfway': [], 'other': [], '22L': [], 'tryLineL':[], 'tryLineR':[], '22R':[]}
    for i in range(len(lines)):
      lineType = str(self.classifyLine(lines[i]))
      linesID += [lineType]
      dist = self.lco.dist(ln=lines[i])
      self.lineTypeLengthSum[lineType] += dist
      lineLengths.append(dist)
      self.lineTypeSlopes[lineType].append(self.lco.getSlope(lines[i]))
    linesID = np.array(list(zip(lines, linesID)),dtype="object")
    for lnType in self.lineTypeSlopes:
     self.lineTypeSlopes[lnType] = np.mean(self.lineTypeSlopes[lnType])
    linesID = self.fixNearCornerUpFieldLines(self.lineTypeSlopes, linesID)
    self.setLineTypeMetrics(linesID)
    linesID = self.lookLeftRight(linesID)
    self.setLineTypeMetrics(linesID)
    linesID = self.differUpPitchLines(linesID)
    self.setLineTypeMetrics(linesID)
    linesID = self.hidegoalPostLines(linesID, self.lineTypeLengthSum)
    self.setLineTypeMetrics(linesID)
    linesID, lineLengths = self.cleanFarSideLine(linesID, lineLengths)
    self.setLineTypeMetrics(linesID)
    linesID = self.cleanHalfwayLine(linesID)
    self.setLineTypeMetrics(linesID)
    linesID = self.detectTryLines(linesID)
    self.setLineTypeMetrics(linesID)
    linesID, lineLengths = self.cleanNearSideLine(linesID, lineLengths)
    self.setLineTypeMetrics(linesID)
    linesID = self.cleanTryLines(linesID)
    return linesID

  def classifyLine(self, ln):
    slope=self.lco.getSlope(ln)
    if np.abs(slope) > 5:
      return "halfway"
    elif  np.isclose(slope, 0, atol=1.9e-1, equal_nan=False):
      return "UpPitchLine"
    elif  np.isclose(slope, -0.6, atol=1.4e-1, equal_nan=False):
      return "22L"
    elif  np.isclose(slope, 0.6, atol=1.4e-1, equal_nan=False):
      return "22R"
    elif slope > -0.5 and slope < 0:
      return "tryLineL"
    elif slope < 0.5 and slope > 0:
      return "tryLineR"
    else:
      return "other"

In [ ]:
class LineDrawer:
  def __init__(self):
    self.lco = LineCornerOperations()

  def listShape(self, lst):
      """
      Recursively get the shape of a nested list.
      """
      if isinstance(lst, list):
          # If lst is a list, recursively get the shape of each element
          inner_shapes = [self.listShape(item) for item in lst]
          # Check if all inner shapes are the same
          if not all(shape == inner_shapes[0] for shape in inner_shapes):
              raise ValueError("Nested list is not rectangular")
          # Return the shape of the first element
          return [len(lst)] + inner_shapes[0]
      else:
          # If lst is not a list, it's a scalar, return an empty list
          return []

  def colorSet(self):
    return {
        "tryLineL"      : (0,0,165),
        "22L"           : (0, 220, 165),
        "halfway"       : (90, 165, 255),
        "22R"           : (165, 0, 0),
        "tryLineR"      : (165, 165, 50),
        "UpPitchLine"   : (255,255,255),
        "nearSideLine"  : (50,200,220),
        "farSideLine"   : (200,220,50),
        "other"         : (0, 165, 90)
    }

  def drawLines(self, image, linesID, color=(0, 0, 0), thickness=2, metrics=False, lineColors=False, cartesianize=False):
    image = np.copy(image)
    # if self.listShape(linesID)[1]==4:
    #   linesID = np.array(list(zip(linesID, (np.full((1,), "other")))))
    for lnData_0 in linesID:
      lnData=np.array(lnData_0, dtype=object)
      ln = lnData[0]
      text = f"{np.round(self.lco.getSlope(ln),2)}" if metrics else ""
      color = self.colorSet()[lnData[1]] if lineColors else color
      colors = []
      pt1,pt2 = self.lco.points(ln)
      cv2.line(image,
                pt1=pt1,
                pt2=pt2,
                color=color,
                thickness=thickness)
      cv2.putText(img=image, text=text, org=(pt1[0]-5, pt1[1]-5),
                  fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.4, color=(0, 0, 255),thickness=1)
      if cartesianize:
        text=str(np.round(lnData[2],2))
        cv2.putText(img=image, text=text, org=(pt2[0]-100, pt2[1]),
                    fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.4, color=(0, 0, 255),thickness=1)
    return image

In [ ]:
class LineCornerOperations:

  def cartesianize(self, ln):
    xs = np.array(ln[0][[0,2]],dtype=np.float64)
    ys = np.array(ln[0][[1,3]], dtype=np.float64)
    return np.polyfit(xs,ys, 1)


  def collinear(self, ln1, ln2, atol=1e-1):
    x1, y1, x2, y2, x3, y3, x4, y4 = ln1[0], ln1[1], ln1[2], ln1[3], ln1[0], ln1[1], ln1[2], ln1[3]
    a = np.abs(x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    b = np.abs(x4 * (y2 - y3) + x2 * (y3 - y4) + x3 * (y4 - y2))
    return  np.isclose(a, 0,  atol=atol) and np.isclose(b, 0, atol=atol)

  def parallel(self, ln1, ln2):
    return (((self.getSlope(ln1) - self.getSlope(ln2))**2)**0.5 <= 10**-10)

  def getSlope(self, ln):
    rise=ln[3] - ln[1]
    run = ln[2] - ln[0]+10**-100
    if run == 0: run=10**-100
    return rise/run

  def joinColLines(self, ln1, ln2):
    xs = np.sort(np.append(ln1, ln2)[::2])
    ys = np.sort(np.append(ln1,ln2)[1::2])
    return [xs[0], ys[0], xs[-1], ys[-1]]

  def getCollinears(self, line, lines):
      colLines = []
      for li in lines:
        ln =  li
        if self.collinear(line, ln):
          colLines.append(ln)
      return colLines

  def points(self, ln):
      x1 = int(ln[0])
      y1 = int(ln[1])
      x2 = int(ln[2])
      y2 = int(ln[3])
      return (x1, y1),(x2, y2)

  def dist(self, pts = (0, 0), ln=[]):
    if not np.array_equal(ln,[]):
      pts = (self.points(ln))
    tupsub = tuple(map(lambda i, j: i - j, pts[0], pts[1]))
    return cv2.norm(tupsub)

  def handleMainLines(self, lines):
    collines = []
    for ln in lines:
      collines.append(self.getColLines(ln, lines))

  def indicesOfUpPitchLine(self, linesID, of=""):
    ids = np.array([row[1] for row in linesID])
    if of=="":
      return np.where((ids == 'UpPitchLine') | (ids == 'nearSideLine'))[0]
    else:
      return np.where((ids == of))[0]

  def refine_lines(self, lines, minDist=20, colLines=120, minLenFactor=0.1):
    keepLines0, keepLines1, colLineCheckLines, colLineNumArr = [], [], [], []
    longestLen = np.max([self.dist(self.points(x)) for x in lines])
    shortestLen = np.min([self.dist(self.points(x)) for x in lines])
    defaultDist = longestLen*minLenFactor
    for li in lines:
        ln =  li
        pt1,pt2 = self.points(ln)
        dist = self.dist((pt1,pt2))
        if dist > defaultDist:
          keepLines0.append(ln)
    return keepLines0

  def lineIntersection(self, ln1, ln2):
    line1 = np.reshape(ln1, (2,-1))
    line2 = np.reshape(ln2, (2,-1))
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])
    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]
    div = det(xdiff, ydiff)
    if div == 0:
      raise Exception('lines do not intersect')
    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return [x, y]

  def findCrossPoints(self, linesID):
    crossPoints = [[],[]]
    try:
        lns = np.asarray(linesID, dtype=object)
        nsl_index = np.where(lns[:, 1] == "nearSideLine")[0]
        nsl = lns[nsl_index[0]] if nsl_index.size > 0 else None

        fsl_index = np.where(lns[:, 1] == "farSideLine")[0]
        fsl = lns[fsl_index[0]] if fsl_index.size > 0 else None

        if nsl is None:
            print("nearSideLine not found")

        if fsl is None:
            print("farSideLine not found")
    except IndexError:
        print("Index error occurred. Check the size of linesID.")
    cross = lns[np.where((  np.invert(lns[:,1]=="nearSideLine")) & ( np.invert(lns[:,1]=="farSideLine")))[0]]
    byLength = np.argsort([-self.dist(ln=ln) for ln in cross[:,0]])
    cross=cross[byLength][0:2]
    crossLineNames = {"tryLineL":0,"22L":1,"halfway":2,"22R":3,"tryLineR":4}
    if crossLineNames[cross[0][1]] > crossLineNames[cross[1][1]]:
      cross=np.flip(cross, axis=0)
    for ln in cross:
      crossPoints[0].append(self.lineIntersection(ln[0], nsl[0]) )
      crossPoints[0].append(self.lineIntersection(ln[0], fsl[0]) )
      crossPoints[1].append(ln[1])
      if len(crossPoints[0]) >=4:
        break
    return crossPoints

In [ ]:
class LineCornerDetector:
  def __init__(self, pd):
    self.ld = LineDrawer()
    self.lco = LineCornerOperations()
    self.lid = LineClassifier(self.lco)
    self.lm = LineMerger(self.lco)
    self.pd=pd


  def lineImagePreprocessing(self, image, show=False):
    image = cv2.convertScaleAbs(image, alpha=1.5, beta=-30)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.convertScaleAbs(gray, alpha=2.85, beta=-240)
    mask = cv2.inRange(image, lowerb=140, upperb=255)
    image = cv2.bitwise_and(image, image, mask=mask)
    if show:
      cv2_imshow(image)
    return image

  def detectLines(self, image):
    dims =np.shape(image)
    startImage = image
    grey = self.lineImagePreprocessing(image)
    lines = np.rot90(
        FastLineDetector(do_merge=False).detect(grey)[0], k=1)
    lines = self.lco.refine_lines(lines, minDist=20)
    return lines

  def detectAndDrawLines(self, startImage, image, color1=(255,255,255), color2=(0,255,0)):
    lines=self.detectLines(image)
    linesID = self.lid.classifyLines(lines)
    image = self.ld.drawLines(startImage, linesID, color=color1, thickness=4, metrics=True, lineColors=True)
    cv2_imshow(image)
    mergedLinesID = self.lm.run(copy.deepcopy(linesID))
    image = self.ld.drawLines(startImage, mergedLinesID, color=color1, thickness=4, metrics=True, lineColors=True, cartesianize=True)
    crossPoints = self.lco.findCrossPoints(mergedLinesID)
    return image, crossPoints

  def run(self, pitchDetector, image, show=True, startImage=None):
    if startImage is None:
      startImage=np.copy(image)
    image, crossPoints = self.detectAndDrawLines(startImage, image, color1=(150,50,50), color2=(255,255,100))
    return image, crossPoints

In [ ]:
class PlayerDetector:
  def __init__(self):
    self.model = model

  def isolatePlayers(self, image):
    mask = np.zeros(shape=np.shape(image)[0:2])
    Grassmask = cv2.inRange(image, (0, 95, 55), (80, 255, 150))
    notGrassmask = cv2.bitwise_not(Grassmask)
    notGrassmask = cv2.dilate(notGrassmask, np.ones((100,100), np.uint8), 3)
    notGrassImg = cv2.bitwise_and(image,  image, mask=notGrassmask)
    return notGrassImg

  def drawRectangles(self, image, dfResults):
    for index, row in dfResults.iterrows():
      imgResult = cv2.rectangle(image, (row['xmin'], row['ymin']), (row['xmax'], row['ymax']), (255, 0, 0), 2)
      imgResult = cv2.putText(image, row['name'],  (row['xmin'], row['ymin']), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    try:
      cv2_imshow(imgResult)
    except:
      print("no players detected")

  def classifyPlayers(self, image, df):
    df['color'] = [(0,100,0) for r in df.iterrows()]
    for i, row in df.iterrows():
      x0,y0,x1,y1 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
      box = image[y0:y1,x0:x1]
      box = self.isolatePlayers(box)
      bsh = np.shape(box)
      box = box[1*bsh[0]//6:3*bsh[0]//6, bsh[1]//3:2*bsh[1]//3]
      boxPxls = cv2.countNonZero(cv2.cvtColor(box, cv2.COLOR_BGR2GRAY))
      jrsys = {
          'England' :[((230,230,230),  (255,255,255)), 1, (255,255,255)],
          'Ref'     :[((0,0,0),           (50,50,50)), 2, (50,50,50)],
          'Bib'     :[((0,	247, 223), (135,255,255)), 2, (135,255,255)],
          'Ireland' :[((23, 70, 5),    (140,255,108)), 2, (1,99,55)]
      }
      pxlCounts=[]
      for j in jrsys:
        dst = cv2.inRange(box, jrsys[j][0][0], jrsys[j][0][1])
        jrsyPxls = cv2.countNonZero(dst)*jrsys[j][1]
        pxlCounts.append(jrsyPxls)
      jr = max(range(len(pxlCounts)), key=pxlCounts.__getitem__)
      df.at[i,'name']=list(jrsys)[jr]
      df.at[i,'color']=list(jrsys.values())[jr][-1]
    return df

  def runDetector(self, image):
    def fixY(y):
      return (y+i*sh[0]//ycut)
    def fixX(x):
      return (x+j*sh[1]//xcut)
    dfResults = pd.DataFrame()
    xcut,ycut=14,2
    sh = np.shape(image)
    for i in range(ycut):
      y0=(sh[0]*i)//ycut
      y1 = sh[0]*(i+1)//ycut
      for j in range(xcut):
        x0 = (sh[1]*j)//xcut
        x1 = (sh[1]*(j+1))//xcut
        box = image[y0:y1,x0:x1]
        results = self.model(box)
        df = results.pandas().xyxy[0]
        df = pd.concat([df, df.loc[(y1-df['ymax']<20) & (df['ymax'] - df['ymin'] < 80)], df.loc[(df['ymin']-y0<20) & (df['ymax'] - df['ymin'] < 80)]]).drop_duplicates(keep=False)
        df = pd.concat([df, df.loc[(x1-df['xmax']<10) & (df['xmax'] - df['xmin'] < 10)], df.loc[(df['xmax']-x0<10) & (df['xmax'] - df['xmin'] < 10)]]).drop_duplicates(keep=False)
        if df.shape[0] > 0:
          df[['xmin','xmax']] = df[['xmin','xmax']].apply(fixX)
          df[['ymin','ymax']] = df[['ymin','ymax']].apply(fixY)
          dfResults = pd.concat([dfResults, df], ignore_index=True)
    results = self.model(image)
    dfResults = pd.concat([dfResults, results.pandas().xyxy[0]], ignore_index=True).drop_duplicates(keep=False)
    dfResults = dfResults.loc[(dfResults['class']==0) & (dfResults['confidence']>0.42)]
    return dfResults

  def getPlayerCoords(self, image):
    # Inference
    image = cv2.convertScaleAbs(image, alpha=1.5, beta=-30)
    dfResults = self.runDetector(image)

    dfResults[['xmin', 'ymin', 'xmax','ymax']] = dfResults[['xmin', 'ymin', 'xmax','ymax']].astype(int)
    dfResults = self.classifyPlayers(image, dfResults)
    dfResults['x'] = (dfResults[['xmin', 'xmax']].mean(axis=1))
    dfResults['y'] = (dfResults['ymax'])
    return np.array([dfResults[['x', 'y', 'name','color']].to_numpy()])

In [ ]:
class PitchTransformer:
  def __init__(self):
    self.lco = LineCornerOperations()
    self.pitchImgShape = (800,1100)

  def drawPitch(self):
    pitch = np.full((self.pitchImgShape[1],self.pitchImgShape[0],3), (119,221,119), dtype=np.uint8)
    color=(240,240,240)
    pitch = cv2.rectangle(pitch, (50,50), (750, 1050), color, 4)
    #50m
    pitch = cv2.line(pitch, pt1=(50,550), pt2=(750,550), color=color, thickness=3)
    pitch = cv2.line(pitch, pt1=(50,830), pt2=(750,830), color=color, thickness=2)
    pitch = cv2.line(pitch, pt1=(50,270), pt2=(750,270), color=color, thickness=2)
    # 10m, 5m across
    pitch = cv2.line(pitch, pt1=(50,450), pt2=(750,450),   color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(50,650), pt2=(750,650),   color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(50,100), pt2=(750,100),   color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(50,1000), pt2=(750,1000), color=(150,150,150), thickness=1)
    # 5m Up, 15m up
    pitch = cv2.line(pitch, pt1=(100,50), pt2=(100,1050), color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(700,50), pt2=(700,1050), color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(200,50), pt2=(200,1050), color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(600,50), pt2=(600,1050), color=(150,150,150), thickness=1)
    return pitch

  def makeImageCorners(self, lineTypes):
    pts = []
    w, l = 750, 1000
    if "tryLineL" in lineTypes:
      pts.append([50,l*0.05])
      pts.append([w,l*0.05])
    if "22L" in lineTypes:
      pts.append([50,l*0.27])
      pts.append([w,l*0.27])
    if "halfway" in lineTypes:
      pts.append([50,l*0.55])
      pts.append([w,l*0.55])
    if "22R" in lineTypes:
      pts.append([50,l*0.83])
      pts.append([w,l*0.83])
    if "tryLineR" in lineTypes:
      pts.append([50,l*1.05])
      pts.append([w,l*1.05])
    pts=np.asarray(pts)
    return np.float32(pts)

  def transformPitch(self, image, crossPoints, playerCoords):
    # cv2_imshow(image)
    shape = np.shape(image)
    pts1 = np.float32(crossPoints[0])
    pts2 = self.makeImageCorners(crossPoints[1])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    image = cv2.warpPerspective(image,M,self.pitchImgShape)
    playerPts = cv2.perspectiveTransform(np.array([playerCoords[0][:,:2]], dtype=np.float32),M)
    try:
      playerCoords[0][:,0], playerCoords[0][:,1] = playerPts[0][:,0], playerPts[0][:,1]
    except Exception as e:
       print( str(e))
    return image, playerCoords

  def projectPlayerCoords(self, playerCoords):
    pitch = self.drawPitch()
    for p in playerCoords[0]:
        pitch = cv2.circle(pitch, (int(p[0]),int(p[1])), radius=10, color=p[3], thickness=-1)
    # print("no players for pitch projection")
    return pitch
    # cv2_imshow(pitch)

  def mergeImageAndProjection(self, image, pitchProjection):
    sh = np.shape(pitchProjection)
    shi = np.shape(image)
    image = cv2.resize(image, (shi[1]*sh[0]//sh[1],sh[0]))
    return cv2.resize(np.concatenate((image, pitchProjection), axis=1), ((2560,1100)))

In [ ]:
class RugbyFieldRegistration:
  def __init__(self):
    self.frameCount=-1
    self.pld = PlayerDetector()
    self.lcd = LineCornerDetector(self)
    self.pt = PitchTransformer()
    self.imo = ImgMorphOperator()
    self.pi = PitchIsolator(self.imo)

  def detectPitch(self, rawFrame):
          self.frameCount+=1
          frame = rawFrame
          frame = self.pi.isolatePitch(frame)
          self.myWriter.cv2Writer.write(copy.deepcopy(cv2.resize(frame, (1600,1100))))
          playerCoords = self.pld.getPlayerCoords(copy.deepcopy(frame))
          # return frame
          frame, crossPoints = self.lcd.run(self, frame, startImage=frame)
          self.myWriter.writeFrame(frame)
          # return frame
          frameTr, playerCoords = self.pt.transformPitch(frame, crossPoints, playerCoords)
          try:
            pitchProjection = self.pt.projectPlayerCoords(playerCoords)
            frame = self.pt.mergeImageAndProjection(frame, pitchProjection)
          except Exception as e:
            print("Failed pitch Transformation\n\n", e)
          return frame

  def skipFrame(self, i, skipFrameRate):
      return not int(i % skipFrameRate)==0

  def flick_through_frames(self, myWriter):
    self.myWriter = myWriter
    i=-1
    while True:
      ret, rawFrame = self.myWriter.cap.read()
      if ret==True:
        i+=1
        if len(self.myWriter.skipFrames)>0 and i in self.myWriter.skipFrames:
          continue
        print(f"frame {i}")
        if i in self.myWriter.skipFrames:
          continue
        if self.myWriter.quitEarly and i > self.myWriter.start_fps:
          break
        frame = self.detectPitch(rawFrame)
        self.myWriter.writeFrame(frame)
      else:
        print("Stream disconnected")
        break

In [ ]:
class VideoWriter:
  def __init__(self):
      self.fourcc = cv2.VideoWriter_fourcc('M','P','4','V')
      self.api=0

  def writeFrame(self, frame):
    self.cv2Writer.write(copy.deepcopy(cv2.resize(frame, (2560,1100))))

  def write_video(self):
    self.cv2Writer = cv2.VideoWriter(
        self.save_path,
        self.api,
        self.fourcc,
        1/2.5,
       (2560,1100))
    self.vpc.flick_through_frames(self)
    self.cv2Writer.release()
    cv2.destroyAllWindows()

  def run(self, cap, save_path, VideoProcessorClass, skipFrames=[], quitEarly=False):
    """
    lnD - lineDetector
    imgPrp - image Preprocessing
    """
    self.vpc = VideoProcessorClass
    self.cap = cap
    self.save_path = save_path
    self.skipFrames=skipFrames
    self.num_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    self.frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    self.frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    self.start_fps = int(cap.get(cv2.CAP_PROP_FPS))
    self.quitEarly = quitEarly
    self.write_video()

In [ ]:
filePath = "/content/drive/My Drive/UL_Masters/rugby/"
vid_cap = cv2.VideoCapture(filePath+"outputs/_40Frames_fullMatch/rawSidOn.mp4")


In [ ]:
skipFrames = [1,5,6,7,8,9,10,13,15,21,22,23,25,26,27,28,29,30,31,32,33,34,36,37,39]
# skipFrames = np.arange(0,38)
pitchDetector = RugbyFieldRegistration()
with warnings.catch_warnings():
  VideoWriter().run(cap=vid_cap, save_path=filePath+"outputs/_40Frames_fullMatch/pitchTransformAndProjection.mp4", VideoProcessorClass=pitchDetector, skipFrames=skipFrames)

In [ ]:
# filePath = "/content/drive/My Drive/UL_Masters/rugby/"
# vid_cap = cv2.VideoCapture(filePath+"outputs/_40Frames_fullMatch/hidePlayers_sidOn.mp4")
# pitchDetector = PitchDetector()
# VideoWriter().run(cap=vid_cap, save_path=filePath+"outputs/_40Frames_fullMatch/y_hidePlayers_do_merge.mp4", VideoProcessorClass=pitchDetector, skipFrames=[1,5,6,7,8,9,10,13,15,21,22,23,25,26,27,28,29,30,31,32,34,36,37,39])

In [ ]:
# vid_cap = cv2.VideoCapture(filePath+"outputs/_40Frames_fullMatch/rawEndOn.mp4")
# pitchDetector = PitchDetector()

# skipFrames=np.append([0,1], np.arange(3,41))
# VideoWriter().run(cap=vid_cap, save_path=filePath+"outputs/_40Frames_fullMatch/yEndOn.mp4", VideoProcessorClass=pitchDetector, skipFrames=[])


## Pitch Isolation